In [3]:
from trainer import *

In [4]:
from ModelBuilder.model_builder import ModelBuilder
from ModelBuilder.poolers import MeanPooling, MaxPooling, ClsPooler

In [9]:
df = pd.read_csv("..\\..\\data\\train_folds.csv")

exp_manager = ExpManager(
    data=df,
    **config
)

exp_manager.model_setup()
exp_manager.fold_trainings()
exp_manager.oof_pred_eval()




##############################
#
#	Fold: 1
#
##############################





KeyboardInterrupt: 

In [10]:
dims = [768*3, 768*2, 768, 512]
ps = [0.2, 0.3, 0.4]
acts = [nn.ReLU(inplace=True)] * (len(dims) - 2) + [None]

In [11]:
[LinBnDrop(i, o, p=p, act=a) for i,o,p,a in zip(dims[:-1], dims[1:], ps, acts)] + [LinBnDrop(dims[-1], 6, bn=False)]

[LinBnDrop(
   (0): BatchNorm1d(2304, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (1): Dropout(p=0.2, inplace=False)
   (2): Linear(in_features=2304, out_features=1536, bias=False)
   (3): ReLU(inplace=True)
 ),
 LinBnDrop(
   (0): BatchNorm1d(1536, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (1): Dropout(p=0.3, inplace=False)
   (2): Linear(in_features=1536, out_features=768, bias=False)
   (3): ReLU(inplace=True)
 ),
 LinBnDrop(
   (0): BatchNorm1d(768, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (1): Dropout(p=0.4, inplace=False)
   (2): Linear(in_features=768, out_features=512, bias=False)
 ),
 LinBnDrop(
   (0): Linear(in_features=512, out_features=6, bias=True)
 )]

In [12]:
model_builder = ModelBuilder(
            model_name="microsoft/deberta-v3-base", 
            dims=dims, 
            ps=ps, 
            poolers=[
                partial(ClsPooler, hidden_size=hidden_size, 
                        last_n_cls=4, drop_p=0),
        MeanPooling,
        MaxPooling
            ], 
            num_labels=6, 
            hidden_dropout_prob=0
        )

In [13]:
model = model_builder.get_model()

In [24]:
def params(m):
    return [p for p in m.parameters()]

def transSplitter(model):
    groups = L(model.body.base_model.embeddings.children())
    for layer in list(model.body.base_model.encoder.layer.children()):
        groups = groups + L(layer)
    groups = groups + L(m for m in list(model.pooler.children()) if params(m)) + L(m for m in list(model.head.children()) if params(m))
    return groups
    # return groups.map(params)

In [31]:
# groups = transSplitter(model)

In [32]:
# groups[14:]

In [33]:
from transformers import AutoTokenizer

In [35]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")

In [40]:
test_toks = tokenizer([
    "this is first text",
    "this is second text"
], padding=True, max_length=1429, return_tensors="pt")

In [41]:
del test_toks["token_type_ids"]

In [42]:
model(**test_toks)

SequenceClassifierOutput(loss=None, logits=tensor([[-0.4600,  0.1987, -0.4446, -0.5276,  0.0577,  0.0620],
        [ 0.1452,  0.1864,  0.2049,  0.1477, -0.3563,  0.0639]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [8]:
list(zip(dims[:-1], dims[1:], ps, acts))

[(2304, 1536, 0.2, ReLU(inplace=True)), (1536, 768, 0.3, None)]